In [1]:
### final code

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [23]:
import time

# get the start time
st = time.time()

urls_ = [
    'https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion',
    'https://www.falabella.com.pe/falabella-pe/category/cat40538/Cocina',
    'https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia',
    'https://www.falabella.com.pe/falabella-pe/category/cat210477/TV-Televisores',
    'https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables',
    'https://www.falabella.com.pe/falabella-pe/category/cat40812/Fotografia',
    'https://www.falabella.com.pe/falabella-pe/category/cat40488/Audio',
    'https://www.falabella.com.pe/falabella-pe/category/cat40556/Videojuegos',
    'https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras',
    'https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home',
    'https://www.falabella.com.pe/falabella-pe/collection/notebook-gamer',
    'https://www.falabella.com.pe/falabella-pe/category/cat40695/Monitores?mkid=LP_MC_MON_22',
    'https://www.falabella.com.pe/falabella-pe/category/cat16470475/Computadores-de-escritorio?mkid=LP_MC_PCG_23',
    'https://www.falabella.com.pe/falabella-pe/category/cat12940610/Audifonos-gamer?mkid=LP_MC_AUD_24',
    'https://www.falabella.com.pe/falabella-pe/category/cat13820483/Sillas-gamer?mkid=LP_MC_SIL_25'
]
"""
urls_ = [
    'https://www.falabella.com.pe/falabella-pe/category/cat40695/Monitores?mkid=LP_MC_MON_22',
    'https://www.falabella.com.pe/falabella-pe/category/cat16470475/Computadores-de-escritorio?mkid=LP_MC_PCG_23',
    'https://www.falabella.com.pe/falabella-pe/category/cat12940610/Audifonos-gamer?mkid=LP_MC_AUD_24',
    'https://www.falabella.com.pe/falabella-pe/category/cat13820483/Sillas-gamer?mkid=LP_MC_SIL_25'
]
"""
ext_url_saga = ['LP_MC_MON_22','LP_MC_PCG_23','LP_MC_AUD_24','LP_MC_SIL_25']
lista_attrs = []
list_discount = []
list_pricecmr = []
list_priceoffer = []
list_pricenormal = []
list_urls = []
list_name = []
a = 0

### get pages per page

def obtainpages(soup):
    pages_ = soup.find('ol',attrs={'class':'jsx-1794558402 jsx-1490357007'}).findAll('li')
    #pages_l = []
    pages_l=[pages_[i].text for i in range(len(pages_))]
    if '...' in pages_l:
        print("si")
        pages_l = list(range(1,int(pages_l[-1])+1))
    return pages_l

def stringtofloat(texto):
    start = texto.find('S/') + 2
    end = len(texto)
    texto = texto[start:end]
    if texto == '':
        texto = '0'
    
    return float(texto.replace(',',''))

for url in urls_:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    number = obtainpages(soup)

    for i in number:
        #print(i)
        try:
            if i != '1':
                #print(url+"?page={}&store=tottus".format(i))
                if 'falabella-pe' in url:
                    if any(ext in url for ext in ext_url_saga):
                        response = requests.get(url+"&page={}".format(i))
                        print(url+"&page={}".format(i))
                    else:
                        response = requests.get(url+"?page={}".format(i))
                        print(url+"?page={}".format(i))
                #elif any(ext in url for ext in ext_url_saga):
                #    response = requests.get(url+"&page={}".format(i))
                #    print(url+"&page={}".format(i))
                else:
                    response = requests.get(url+"?page={}&store=tottus".format(i))
                    print(url+"?page={}&store=tottus".format(i))
                #print(response)
                soup = BeautifulSoup(response.content, "html.parser")

            #lista de productos a scrappear
            lista_ = soup.find_all("div",class_="jsx-4099777552 search-results--products")[0].find_all("div",class_="jsx-1327784995 jsx-97019337 pod pod-4_GRID")

            for list_ in lista_:
                a = a+1
                #print(list_.find("ol").find_all("span"))
                lista_n = list_.find("ol").find_all("span")

                #lista_c = lista_[18]
                url_ = list_.find("a")["href"]
                try:
                    name = list_.find("img")["alt"].strip()
                    #print()
                except:
                    name = list_.find_all("b")[1].text

                for x in range(len(lista_n)):
                    #print(lista_n[x])
                    value = lista_n[x].text.strip()

                    if len(lista_n) == 1:
                        #print("es uno")
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','medium']):
                            price_o = lista_n[x].text.strip()
                            #print("PO: "+price_o)
                        price_n = ''
                        price_cmr = ''
                        discount = 0
                    if len(lista_n) == 2:
                        #print("es dos")
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','medium']):
                            price_o = lista_n[x].text.strip()
                            #print("PO: "+price_o)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy3','medium']):
                            price_n = lista_n[x].text.strip()
                            #print("PN: "+price_n)
                        price_cmr = ''
                        if price_n != '':
                            discount = int(str(round((1-(stringtofloat(price_o)/stringtofloat(price_n)))*100)))
                        else:
                            discount = 0

                    if len(lista_n) == 3:
                        #print("es tres")
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','medium']):
                            price_o = lista_n[x].text.strip()
                            #print("PO: "+price_o)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy3','medium']):
                            price_n = lista_n[x].text.strip()
                            #print("PN: "+price_n)
                        price_cmr = ''
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy5','primary']):
                            discount = -1*int(lista_n[x].text.strip().replace('%',''))
                            #print("DS: "+discount)

                    if len(lista_n) == 4:
                        #print("es cuatro")
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','medium']):
                            price_o = lista_n[x].text.strip()
                            #print("PO: "+price_o)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy3','medium']):
                            price_n = lista_n[x].text.strip()
                            #print("PN: "+price_n)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','high']):
                            price_cmr = lista_n[x].text.strip()
                            #print("PC: "+price_cmr)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy5','primary']):
                            discount = -1*int(lista_n[x].text.strip().replace('%',''))
                            #print("DS: "+discount)
            
            list_pricecmr.append(price_cmr)
            list_discount.append(discount)
            list_priceoffer.append(price_o)
            list_pricenormal.append(price_n)
            list_urls.append(url_)
            list_name.append(name)
        except:
            print("can't not read the page")
            pass
            traceback.print_exc()

# get the end time
et = time.time()

# get the execution time
res = et - st
final_res = res / 60
print('Execution time:', final_res, 'minutes')

si
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=1
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=2
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=3
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=4
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=5
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=6
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=7
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=8
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=9
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=10
https://www.falabella.com.pe/falabella-pe/category/CATG19032/Refrigeracion?page=11
si
https://www.falabella.com.pe/falabella-pe/category/cat40538/Cocina?page=1
https://www.fala

https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=66
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=67
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=68
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=69
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=70
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=71
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=72
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=73
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=74
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=75
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=76
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=77
https://www.falabella.com.pe/falabella-pe/category/c

https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=169
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=170
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=171
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=172
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=173
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=174
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=175
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=176
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=177
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=178
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=179
https://www.falabella.com.pe/falabella-pe/category/cat760702/Telefonia?page=180
https://www.falabella.com.pe/falabella-p

Traceback (most recent call last):
  File "/tmp/ipykernel_1316/2924442232.py", line 88, in <module>
    lista_ = soup.find_all("div",class_="jsx-4099777552 search-results--products")[0].find_all("div",class_="jsx-1327784995 jsx-97019337 pod pod-4_GRID")
IndexError: list index out of range


si
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=1
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=2
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=3
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=4
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=5
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=6
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=7
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=8
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=9
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-wearables?page=10
https://www.falabella.com.pe/falabella-pe/category/cat1830468/Smartwatch-y-w

...

https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=127
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=128
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=129
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=130
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=131
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=132
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=133
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=134
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=135
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=136
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=137
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=138
https://www.fala

Traceback (most recent call last):
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 1042, in _validate_conn
    conn.connect()
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/connection.py", line 414, in connect
    self.sock = ssl_wrap_socket(
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "/usr/lib/python3.10/ssl.py", line 513, in wrap_socket
    retu

https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=184
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=185
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=186
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=187
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=188
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=189
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=190
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=191
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=192
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=193
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=194
https://www.falabella.com.pe/falabella-pe/category/cat50678/Computadoras?page=195
https://www.fala

Traceback (most recent call last):
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/connectionpool.py", line 1042, in _validate_conn
    conn.connect()
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/connection.py", line 414, in connect
    self.sock = ssl_wrap_socket(
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "/home/nitram/.local/lib/python3.10/site-packages/urllib3/util/ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "/usr/lib/python3.10/ssl.py", line 513, in wrap_socket
    retu

https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=61
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=62
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=63
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=64
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=65
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=66
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=67
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=68
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=69
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotica-y-Smart-Home?page=70
https://www.falabella.com.pe/falabella-pe/category/cat11190471/Domotic

In [15]:
import traceback

try:
    raise TypeError("Oups!")
except:
    try:
        raise TypeError("Again !?!")
    except:
        pass

    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_1316/2522101750.py", line 4, in <module>
    raise TypeError("Oups!")
TypeError: Oups!


In [24]:
import csv
df.to_csv('test.txt', index=False, quotechar='"',
                      header=None, quoting=csv.QUOTE_NONNUMERIC)
df = pd.DataFrame({'Product Name':list_name,'Price_cmr':list_pricecmr,'Price_offer':list_priceoffer,'Price_normal':list_pricenormal, 'Discount':list_discount, 'links':list_urls})
df = df.query("`Discount` >= 50").sort_values("Discount", ascending=False)
df = df.drop_duplicates()
df

,Product Name,Price_cmr,Price_offer,Price_normal,Discount,links
602,Parlante Bluetooth 8W Recargable,,S/ 69,"S/ 1,899",96,https://sodimac.falabella.com.pe/sodimac-pe/pr...
353,Mini Trípode Soporte Para Celular 360°,,S/ 11.90,S/ 162,93,https://www.falabella.com.pe/falabella-pe/prod...
439,PARLANTE BLACKSHEEP DOME BLACK + HIELERA METALI,,S/ 89,S/ 298,70,https://www.falabella.com.pe/falabella-pe/prod...
711,Camara web c270 logitech 3mp 1280 x 720 pixele...,,S/ 117.89,S/ 399,70,https://www.falabella.com.pe/falabella-pe/prod...
331,Correa Silicona para Smart Watch 7 y 8 Color N...,,S/ 13,S/ 40,68,https://www.falabella.com.pe/falabella-pe/prod...
...,...,...,...,...,...,...
459,Auriculares Bluetooth 7+14H Live Free NC+,,S/ 347,S/ 699,50,https://www.falabella.com.pe/falabella-pe/prod...
220,Mica para Motorola E7 Protector de Pantalla 21...,,S/ 19.89,S/ 39.90,50,https://www.falabella.com.pe/falabella-pe/prod...
393,Audífonos Inalámbricos Airpods I7S TWS - Blanco,,S/ 24.99,S/ 49.99,50,https://www.falabella.com.pe/falabella-pe/prod...
303,Funda Forro Protector Control Magic LG Nueva V...,,S/ 29.99,S/ 59.99,50,https://www.falabella.com.pe/falabella-pe/prod...


In [27]:
import csv
df.to_csv('products_saga.csv', sep=',', index=False, encoding='utf-8',quotechar='"', header=None, quoting=csv.QUOTE_NONNUMERIC)

In [19]:
url = 'https://www.falabella.com.pe/falabella-pe/category/cat40488/Audio?page=10'

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
lista_ = soup.find_all("div",class_="jsx-4099777552 search-results--products")[0].find_all("div",class_="jsx-1327784995 jsx-97019337 pod pod-4_GRID")
lista_

[<div class="jsx-1327784995 jsx-97019337 pod pod-4_GRID" data-category="J11021301" data-gsccategory="" data-key="prod16710019" data-pod="catalyst-pod" data-sponsored="true" id="testId-pod-18049547" role="button" tabindex="0"><div class="jsx-1327784995 jsx-97019337 pod-head"><div class="jsx-2907167179"><a class="jsx-2907167179 layout_grid-view layout_view_4_GRID" href="https://www.falabella.com.pe/falabella-pe/product/prod16710019/Audifonos-Skullcandy-Dime-True-wireless/18049547"><picture class="jsx-1996933093"><source class="jsx-1996933093" srcset="https://falabella.scene7.com/is/image/FalabellaPE/18049547_1?wid=240&amp;hei=240&amp;qlt=70&amp;fmt=webp, https://falabella.scene7.com/is/image/FalabellaPE/18049547_1?wid=480&amp;hei=480&amp;qlt=70&amp;fmt=webp 2x" type="image/webp"><img alt="SKULLCANDY - Audífonos Skullcandy Dime True wireless" class="jsx-1996933093" height="240" id="testId-pod-image-https://falabella.scene7.com/is/image/FalabellaPE/18049547_1" src="https://falabella.scene7

In [20]:
for list_ in lista_:
                a = a+1
                #print(list_.find("ol").find_all("span"))
                lista_n = list_.find("ol").find_all("span")

                #lista_c = lista_[18]
                url_ = list_.find("a")["href"]
                try:
                    name = list_.find("img")["alt"].strip()
                    #print()
                except:
                    name = list_.find_all("b")[1].text

                for x in range(len(lista_n)):
                    #print(lista_n[x])
                    value = lista_n[x].text.strip()

                    if len(lista_n) == 1:
                        #print("es uno")
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','medium']):
                            price_o = lista_n[x].text.strip()
                            #print("PO: "+price_o)
                        price_n = ''
                        price_cmr = ''
                        discount = 0
                    if len(lista_n) == 2:
                        #print("es dos")
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','medium']):
                            price_o = lista_n[x].text.strip()
                            print("PO: "+price_o)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy3','medium']):
                            price_n = lista_n[x].text.strip()
                            print("PN: "+price_n)
                        price_cmr = ''
                        if price_n != '':
                            discount = int(str(round((1-(stringtofloat(price_o)/stringtofloat(price_n)))*100)))
                        else:
                            discount = 0

                    if len(lista_n) == 3:
                        #print("es tres")
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','medium']):
                            price_o = lista_n[x].text.strip()
                            #print("PO: "+price_o)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy3','medium']):
                            price_n = lista_n[x].text.strip()
                            #print("PN: "+price_n)
                        price_cmr = ''
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy5','primary']):
                            discount = -1*int(lista_n[x].text.strip().replace('%',''))
                            #print("DS: "+discount)

                    if len(lista_n) == 4:
                        #print("es cuatro")
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','medium']):
                            price_o = lista_n[x].text.strip()
                            #print("PO: "+price_o)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy3','medium']):
                            price_n = lista_n[x].text.strip()
                            #print("PN: "+price_n)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy10','high']):
                            price_cmr = lista_n[x].text.strip()
                            #print("PC: "+price_cmr)
                        if all(any(m in y for y in lista_n[x]["class"]) for m in ['copy5','primary']):
                            discount = -1*int(lista_n[x].text.strip().replace('%',''))
                            #print("DS: "+discount)

PO: S/  779
PN: S/  849
PO: S/  1,399
PN: S/  1,499 - S/  1,599


ValueError: could not convert string to float: '  1499 - S/  1599'